The data obtained on this csv is coming form this query obtaining the results from the CDW
This notebook is to aggregate it and organize it in a way that can be use later on the analysis


```
select
   wsf.date,
   wpr.websession_landing_url_raw as URL,
   wpd.page_no_query_string as URLClean,
   tsd.traffic_source as Source,
   count(distinct wsf.websession_id) as Sessions,
   sum(wsf.total_pages_viewed) as Pageviews,
   sum(wsf.return_visitor) as Return
INTO platea_sessions
from cdm.warehouse.web_session_fct AS wsf
   inner join cdm.warehouse.websession_landing_webpage_raw_dim wpr using (websession_landing_webpage_raw_dk)
   inner join cdm.warehouse.webpage_dim wpd USING (webpage_dk)
   inner join cdm.warehouse.traffic_source_dim AS tsd USING (traffic_source_dk)
WHERE   wsf.tenantid = 'src_1qnv2ee86cv3SDVqJk0YUhK40mN'

AND (wpr.websession_landing_url_raw LIKE 'https://www.plateapr.com%' or wpr.websession_landing_url_raw LIKE 'https://mapa.plateapr.com%')
 AND wsf.is_botsession_ind != 1
GROUP BY
        1,2,3,4
ORDER BY
        1
 ```

In [3]:
import pandas as pd

In [4]:
data = pd.read_csv("./data/platea_sessions.csv", names=['date', 'url', 'url_clean', 'source', 'sessions', 'views', 'return_visitors'] )

In [9]:
data.head()

,date,url,url_clean,source,sessions,views,return_visitors
0,2022-02-06,https://www.plateapr.com/lugares-donde-para-vi...,www.plateapr.com/lugares-donde-para-visitar-cu...,ORGANIC,1,1,0
1,2022-02-06,https://www.plateapr.com/trails-en-puerto-rico...,www.plateapr.com/trails-en-puerto-rico-para-ha...,DIRECT,1,1,0
2,2022-02-06,https://www.plateapr.com/sign-up/?utm_source=i...,www.plateapr.com/sign-up/,PAID,1,1,0
3,2022-02-06,https://www.plateapr.com/sign-up/?utm_source=i...,www.plateapr.com/sign-up/,PAID,1,1,1
4,2022-02-06,https://www.plateapr.com/restaurantes-para-dis...,www.plateapr.com/restaurantes-para-disfrutar-e...,PAID,1,1,1


In [23]:
data['url_clean'] = ['https://' + url if url.startswith('www.plateapr.com') else url for url in data['url_clean'] ]

In [26]:
to_save = data.groupby(['url_clean']).sum()
to_save.to_csv('./data/sessions_sum.csv')

In [27]:
data['url_clean'].value_counts()

https://www.plateapr.com/que-hacer-este-fin-de-semana-en-puerto-rico/        10581
mapa.plateapr.com/places                                                      4631
https://www.plateapr.com/sign-up/                                             3000
https://www.plateapr.com/app/callback/                                        2779
https://www.plateapr.com/                                                     2419
                                                                             ...  
https://www.plateapr.com/search/restaurantes con vista al mar en san juan        1
https://www.plateapr.com/search/ocean view                                       1
https://www.plateapr.com/tag/hiking/                                             1
https://www.plateapr.com/search/cervesa                                          1
https://www.plateapr.com/search/san sebastián                                    1
Name: url_clean, Length: 1244, dtype: int64

In [28]:
to_save = data.groupby(['url_clean']).mean()
to_save.to_csv('./data/sessions_mean.csv')

In [5]:
data['url_clean'].value_counts()

www.plateapr.com/que-hacer-este-fin-de-semana-en-puerto-rico/        10581
mapa.plateapr.com/places                                              4631
www.plateapr.com/sign-up/                                             3000
www.plateapr.com/app/callback/                                        2779
www.plateapr.com/                                                     2419
                                                                     ...  
www.plateapr.com/search/restaurantes con vista al mar en san juan        1
www.plateapr.com/search/ocean view                                       1
www.plateapr.com/tag/hiking/                                             1
www.plateapr.com/search/cervesa                                          1
www.plateapr.com/search/san sebastián                                    1
Name: url_clean, Length: 1244, dtype: int64

In [10]:
data.groupby('url_clean').agg({'sessions': 'sum', 'views':'sum', 'return_visitors':'sum'}).reset_index().sort_values(by=['sessions'], ascending=False)[0:30]

,url_clean,sessions,views,return_visitors
264,www.plateapr.com/,50215,196765,17534
694,www.plateapr.com/que-hacer-este-fin-de-semana-...,35055,87706,10516
4,mapa.plateapr.com/places,16199,20908,8477
575,www.plateapr.com/en/travel/puerto-rico-declara...,13560,18813,1789
415,www.plateapr.com/en/food-drinks/best-brunch-pu...,11263,13314,2221
551,www.plateapr.com/en/things-to-do/best-hikes-in...,9931,11538,2370
418,www.plateapr.com/en/food-drinks/san-juan-resta...,9101,11116,1617
709,www.plateapr.com/restaurantes-con-vista-a-las-...,8899,14987,2276
393,www.plateapr.com/en/,7884,38196,2223
1229,www.plateapr.com/unete-a-platea/,7411,31805,2458


In [ ]:
df.sort_values(by=['col1'])